In [7]:
import os

import pandas as pd
import numpy as np

from env import user, password, host

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings("ignore")

## Data Dictionary

In [2]:
data_dict = pd.read_excel('zillow_data_dictionary.xlsx')
data_dict.sort_values(by = 'Feature')

,Feature,Description
0,'airconditioningtypeid',Type of cooling system present in the home (i...
1,'architecturalstyletypeid',"Architectural style of the home (i.e. ranch, ..."
55,'assessmentyear',The year of the property tax assessment
2,'basementsqft',Finished living area below or partially below...
3,'bathroomcnt',Number of bathrooms in home including fractio...
4,'bedroomcnt',Number of bedrooms in home
6,'buildingclasstypeid',"The building framing type (steel frame, wood f..."
5,'buildingqualitytypeid',Overall assessment of condition of the buildi...
7,'calculatedbathnbr',Number of bathrooms in home including fractio...
11,'calculatedfinishedsquarefeet',Calculated total finished living area of the ...


# Acquire
Acquire 
- `bedroomcnt`, 
- `bathroomcnt`, 
-`calculatedfinishedsquarefeet`, 
- `taxvaluedollarcnt`, 
- `yearbuilt`, 
- `taxamount`, and 
- `fips`

from the zillow database for all 'Single Family Residential' properties.

### Function to connect to Codeup mysql server

In [3]:
def get_connection(database_name):
    '''
    This function takes in a database name from the sequel server and returns a string with 
    the credentials needed to access that server.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{database_name}'

### Function to acquire data from server
1. created a variable to hold query
2. can be updated or modified if want to make changes later

In [4]:
# 1. query used to acquire data from sql

query = '''select bedroomcnt,
    bathroomcnt,
    calculatedfinishedsquarefeet,
    taxvaluedollarcnt,
    yearbuilt,
    taxamount,
    fips
    from properties_2017
    where propertylandusetypeid = 261;'''

In [5]:
# 2. function to acquire data (uses the query)

def get_zillow_data():
    '''
    This function reads the zillow database from the Codeup mysql server adn returns a dataframe
    If a local csv file does not exist, this function writes the data to a csv file
    The local file can be used as a backup in the event that you cannot access the mysql database
    '''
    
    
    if os.path.isfile('zillow.csv'):
# if a local file already exists
        df = pd.read_csv('zillow.csv', index_col = 0)
        
    else:
# if a local file does not yet exist
        df = pd.read_sql(query, get_connection('zillow'))

# and cache data into csv file
        df.to_csv('zillow.csv')
    
    return df

### Taking a look at the data

In [8]:
# peak at the data

zillow = get_zillow_data()
zillow.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


# 2. Summarization and Cleaning

In [9]:
print(f'Rows: {zillow.shape[0]}\nColumns: {zillow.shape[1]}')

Rows: 2152863
Columns: 7


In [10]:
zillow.describe().T

,count,mean,std,min,25%,50%,75%,max
bedroomcnt,2152852.0,3.287196,0.954754,0.00,3.00,3.00,4.00,25.00
bathroomcnt,2152852.0,2.230688,0.999280,0.00,2.00,2.00,3.00,32.00
calculatedfinishedsquarefeet,2144379.0,1862.855178,1222.125124,1.00,1257.00,1623.00,2208.00,952576.00
taxvaluedollarcnt,2152370.0,461896.237963,699676.049600,1.00,188170.25,327671.00,534527.00,98428909.00
yearbuilt,2143526.0,1960.949681,22.162196,1801.00,1949.00,1958.00,1976.00,2016.00
taxamount,2148421.0,5634.865978,8178.910249,1.85,2534.98,4108.95,6414.32,1337755.86
fips,2152863.0,6048.377335,20.433292,6037.00,6037.00,6037.00,6059.00,6111.00


### Handling our null values

In [11]:
zillow.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2152852 non-null  float64
 1   bathroomcnt                   2152852 non-null  float64
 2   calculatedfinishedsquarefeet  2144379 non-null  float64
 3   taxvaluedollarcnt             2152370 non-null  float64
 4   yearbuilt                     2143526 non-null  float64
 5   taxamount                     2148421 non-null  float64
 6   fips                          2152863 non-null  float64
dtypes: float64(7)
memory usage: 131.4 MB


In [ ]:
# percentage of null values in each column to total values

print(f'{(zillow.isnull().sum()) / (zillow.shape[0])}')


### ----> commented out: 
- We have such a small percentage of nulls compared to total that we can drop those rows

In [ ]:
# # dropping null rows

# zillow.dropna(inplace = True)

In [ ]:
# confirming no null values in df

zillow.isna().sum()

### Cleaning up our column names and data types

In [ ]:
# looking at the values for each column and creating a cols list
cols = []

for col in zillow.columns:
    cols.append(col)
    print(col)
    print(zillow[col].value_counts())
    print('------------------------')
    print()

In [ ]:
cols

In [ ]:
# renaming columns

zillow.rename(columns = {'bedroomcnt': 'bedrooms',
                         'bathroomcnt': 'bathrooms',
                         'calculatedfinishedsquarefeet': 'sqft',
                         'taxvaluedollarcnt': 'tax_value',
                         'yearbuilt': 'year_built',
                         'taxamount': 'tax_amt',
                         'fips': 'fips'}, inplace = True)

In [ ]:
# preparing for loop that converts floats ------> to integers by creating 2 lists from columns
cols = zillow.columns
cols = cols.to_list()
print(f'All columns: {cols}\n')

flt_cols = ['bathrooms', 'sqft', 'tax_value', 'tax_amt']
int_cols = ['bedrooms', 'year_built', 'fips']
print(f'Float columns: {flt_cols}')
print(f'Integer columns: {int_cols}')

### Conversion to integer commented out for now because of nulls

In [ ]:
# # for loop to convert floats to -----> int

# for col in int_cols:
#     zillow[col] = zillow[col].astype(int)

In [ ]:
# confirming our dtypes have been updated

zillow.dtypes

https://github.com/CodeupClassroom/germain-regression-exercises/blob/main/wrangle_exercise_review.ipynb
<br><i>referenced for loop from Germain regression wrange_exercises_review.ipynb solutions</i>

In [ ]:
cols

In [ ]:
# variable distributions

plt.figure(figsize = (16, 3))

# list of columns
cols = [col for col in zillow.columns if col not in ['fips', 'year_built']]

for i, col in enumerate(cols):
    # i starts at 0, but I want the plot number to start at 1 ---> adding 1
    plot_number = i + 1

    # create subplots    
    plt.subplot(1, len(cols), plot_number)
    
    # title with column name
    plt.title(col)
    
    # creating histogram columns
    zillow[col].hist(bins = 5)

    # hiding gridlines
    plt.grid(False)

plt.show()

In [ ]:
# boxplots
plt.figure(figsize = (8, 4))

plt.ticklabel_format(useOffset = False, style = 'plain')
sns.boxplot(data = zillow.drop(columns = ['fips']))

plt.show()

### There are a lot of outliers in the dataset that need to be removed prior to exploration
- We can use the iqr rule to remove outliers

In [ ]:
zil_cols = zillow.columns.to_list()

In [ ]:
# function to remove outliers
def remove_outliers(df, k, cols):
    '''
    This function uses the iqr rule to remove outliers from a list of columns in a DataFrame.
       - Args:
            - df: DataFrame
            - k: upper and lower bound +/- value
            - cols: DataFrame columns (want outliers removed from)
        - Returns:
            - DataFrame with outliers removed from chosen columns
    '''
    
    for col in cols:
        
    #     get quartiles
        q1, q3 = df[col].quantile([.25, .75])

    #     calculate interquartile range (iqr range)
        iqr = q3 - q1

    #     get upper and lower bounds
        upper_bound = q3 + k * iqr
        lower_bound = q1 - k * iqr

    #     return dataframe without outliers
        df = df[(df[col] > lower_bound) & (df[col] < upper_bound)]
    
    return df

In [ ]:
zillow_no_outliers = remove_outliers(zillow, 1.5, cols)
zillow_no_outliers.head()

### Taking a look at the distribution

In [ ]:
print(f'zil_cols: {zil_cols}')
print()
print(f'cols: {cols}')

In [ ]:
# histograms
plt.figure(figsize = (16, 3))

for i, col in enumerate(cols):
    
#     i starts at 0, but plot numbers should start at 1
    plot_number = i + 1
    
#     create subplot
    plt.subplot(1, len(cols), plot_number)
    
#     title as column name
    plt.title(col)
    
#     display histograms from each column
    zillow_no_outliers[col].hist(bins = 5)
    
#     hide gridlines
    plt.grid(False)
    
#     add padding to prevent plot overlap
    plt.tight_layout()

In [ ]:
# boxplots
plt.figure(figsize = (16, 3))

for i, col in enumerate(cols):
#     i starts at 0, but plot should start at 1
    plot_number = i + 1
    
#     create subplot
    plt.subplot(1, len(cols), plot_number)
    
#     title with column names
    plt.title(col)
    
#     display boxplots for column
    sns.boxplot(data = zillow_no_outliers[[col]])
    
    

### Summary Takeaways:
>`bedrooms` and `bathrooms` are not normally distributed

# Prepare
- Are there any null values?

In [ ]:
zillow_no_outliers.isna().sum()

### Imputing year_built

In [ ]:
zillow_no_outliers.year_built.value_counts()

##### Because of the small number of missing values, will impute year_build after split

### Looking at the data types

In [ ]:
zillow_no_outliers.info()

In [ ]:
# looking at the values to determine which, if any, should be converted from float to int
for col in zillow_no_outliers.columns:
    print(col.upper())
    print(zillow_no_outliers[col].value_counts())
    print('--------------------')
    print()

### will cast fips and year_built as objects bc they are currently numerical representations of categorical values

In [ ]:
zillow_no_outliers.fips = zillow_no_outliers.fips.astype(object)
zillow_no_outliers.year_built = zillow_no_outliers.year_built.astype(object)

In [ ]:
zillow_no_outliers.info()

## Do my columns contain a target leak?
>Model Goal:<br>
build a ML model that can predict a property's value<br>

`tax_amt` is determined by a real estate tax appraiser's valuation of the home, combined with the local tax rate<br>

### Target Leak:
>Does this indirectly represent the target variable?<br>
Does this represent information that we wouldn't have at the time we use our model to predictions on new data?


Reasons a tax appraiser's value may deviate from from the sale price of a property:
- inversely related incentives (incentive to reduce `tax appraised value` but to increase `sale price property value`)
- Counties may limit `tax appraised value` increase to a set amt, causing them to lag behind `home values` in a hot market
- local tax appraisers evaluate homes based on evaluation criteria that can differ from county to county or year to year based on political and administrative pressures

#### For now we will leave `tax_amt` in the data set as it is not clear that it indirectly represents the target variable. 

# Split the Data

In [ ]:
train_validate, test = train_test_split(zillow_no_outliers, test_size = .2, random_state = 123)
train, validate = train_test_split(train_validate, test_size = .3, random_state = 123)

# Impute missing year_build values with mode

In [ ]:
# build imputer
imputer = SimpleImputer(strategy = 'median')

# fit to train
imputer.fit(train[['year_built']])

# transform the data using imputed values
train[['year_built']] = imputer.transform(train[['year_built']])
validate[['year_built']] = imputer.transform(validate[['year_built']])
test[['year_built']] = imputer.transform(test[['year_built']])

In [ ]:
# for loop to verify missing values imputed in all three datasets

for df in train, validate, test:
    print(df.isna().sum())
    print('-----------------------')
    print()

# for final wrangle.py file
a. Imports<br>
b. `acquire_zillow`( ) function<br>
   >- b-01. read queried data into df
   >- b-02. rename columns

c. `remove_outliers`(df, k, col_list) function<br> 
d. data visualizations<br>
>- d-01. `get_hist`(df) histograph function
>- d-02. `get_box`(df) boxplots function

e. composite `wrangle_zillow`(df) function<br>
<i>composite of c and d</i>
>- e-01. remove outliers
>- e-02. visualizations to get distributions of numeric data
>    - e-02a. `get_hist` histographs
>    - e-02b. `get_box` boxplots
>- e-03. convert `fips` and `year_built` columns to objects
>- e-04. train/valiate/test split
>- e-05. impute year using median

## Final `wrangle_zillow`( ) function
Takes in the above functions and code and returns three DataFrames, prepared for exploration